In [1]:
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import resample
from sklearn import preprocessing
import statistics
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, log_loss, average_precision_score
from random import sample, randint, randrange
from copy import deepcopy
from sklearn.model_selection import cross_validate

In [2]:
from contextlib import contextmanager
from timeit import default_timer

@contextmanager
def elapsed_timer():
    start_time = default_timer()

    class _Timer():
      start = start_time
      end = default_timer()
      duration = end - start

    yield _Timer

    end_time = default_timer()
    _Timer.end = end_time
    _Timer.duration = end_time - start_time

In [3]:
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')

In [4]:
vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(newsgroups_train.data)
y_train = newsgroups_train.target
x_test = vectorizer.transform(newsgroups_test.data)
y_test = newsgroups_test.target
#x_train = np.concatenate([x_train, x_test])
#y_train = np.concatenate([y_train, y_test])
classes_count = 20

Change dimensions to 500

In [5]:
svd = TruncatedSVD(n_components=500)
svd.fit(x_train)
svd_x_train = svd.transform(x_train)
svd_x_test = svd.transform(x_test)

In [5]:
#x_train_res, y_train_res = resample(svd_x_train, y_train, n_samples=5000, replace=False, random_state=0)
#x_test_res, y_test_res = resample(svd_x_test, y_test, n_samples=500, replace=False, random_state=0)
#x_train_res = x_train_res.reshape((x_train_res.shape[0],-1))
#x_test_res = x_test_res.reshape((x_test_res.shape[0],-1))
#y_train_res = y_train_res.reshape((y_train_res.shape[0],))

In [6]:
#scaler = StandardScaler(with_mean=False).fit(x_train)
#x_train_scaled = scaler.transform(x_train) # scaling data
#x_test_scaled = scaler.transform(x_test)
#print(x_train_scaled.shape)

SVM on full data and full features

In [6]:
clf = SVC(probability=True)
clf.fit(svd_x_train, y_train)
#clf.fit(x_train_res, y_train_res)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [7]:
y_pred = clf.predict(svd_x_test)
#y_pred = clf.predict(x_test_res)

In [8]:
accuracy = accuracy_score(y_test, y_pred)
#accuracy = accuracy_score(y_test_res, y_pred)

In [9]:
accuracy #0.78956

0.7896972915560276

In [10]:
f_score = f1_score(y_test, y_pred, average='weighted')
#f_score = f1_score(y_test_res, y_pred, average='weighted')
f_score

0.7909724277855422

10 sub-classifiers

In [27]:
n_samples = [0.1, 0.25]
n_features = [0.25, 0.5, 0.75]
n_classifiers = 6

samples_all = svd_x_train.shape[0]
samples_all_test = svd_x_test.shape[0]
features_all = svd_x_train.shape[1]

In [189]:
class MinorClassifiers:
    def __init__(self, n_samples, n_features):
        self.n_samples = n_samples
        self.n_features = n_features
        self.classifiers = []
        self.predictions = []
        self.cut_features = []
    
    def get_params(self, deep = False):
        return {
            'n_samples': self.n_samples,
            'n_features': self.n_features,   
        }
    
    def predict(self, X):
        if self.predictions == []:
            for i in range(len(self.classifiers)):
                classifier = self.classifiers[i]
                x_test = X[:,self.cut_features[i]]
                y_pred = classifier.predict(x_test)
                pred = classifier.predict_proba(x_test)
                self.predictions.append((y_pred, pred))
            
        return majority_pred(self.predictions)
    
    def predict_proba(self, X):
        if self.predictions == []:
            for i in range(len(self.classifiers)):
                classifier = self.classifiers[i]
                x_test = X[:,self.cut_features[i]]
                y_pred = classifier.predict(x_test)
                pred = classifier.predict_proba(x_test)
                self.predictions.append((y_pred, pred))
        return majority_pred_proba(self.predictions, len(self.classifiers))
        
    def fit(self, X, Y):
        feature_list = [n for n in range(500)]
        samples_all = X.shape[0]
        features_all = X.shape[1]

        for samples in self.n_samples:
            for features in self.n_features:
                f = sample(feature_list, int(features_all * features))
                self.cut_features.append(f)
                x_train_f = X[:,f]
                
                x_train_s, y_train_s = resample(x_train_f, Y, n_samples=int(samples * samples_all), replace=False, random_state=0)

                svm_clf = SVC(probability=True)
                svm_clf.fit(x_train_s, y_train_s)
                
                self.classifiers.append(svm_clf)

In [13]:
def get_minor_classifiers(samples_perc, features_perc):
    feature_list = [n for n in range(500)]
    classifiers = []
    
    for samples in samples_perc:
        for features in features_perc:
            f = sample(feature_list, int(features_all * features))
            x_train_f = svd_x_train[:,f]
            x_test_f = svd_x_test[:,f]
            
            x_train_s, y_train_s = resample(x_train_f, y_train, n_samples=int(samples * samples_all), replace=False, random_state=0)
            #x_test_s, y_test_s = resample(svd_x_test, y_test, n_samples=int(samples * samples_all_test), replace=False, random_state=0)
            
            svm_clf = SVC(probability=True)
            svm_clf.fit(x_train_s, y_train_s)
            
            y_pred = svm_clf.predict(x_test_f)
            pred = svm_clf.predict_proba(x_test_f)
            
            classifiers.append((y_pred, pred))
            
    return classifiers

In [14]:
#clas = get_minor_classifiers(n_samples, n_features)

In [192]:
def average_pred(predictions):
    #predictions = list of tuples 
    m = len(predictions[0][0])
    all_results = [[0 for x in range(classes_count)] for y in range(m)] 
    results = [0] * m
    for (_, pred_proba) in predictions:
        for i in range(m):
            for j in range(classes_count):
                all_results[i][j] += pred_proba[i][j]
    for i in range(m):
        results[i] = all_results[i].index(max(all_results[i]))
    return results

def average_pred_proba(predictions, n_classifiers):
    m = len(predictions[0][0])
    results = [[0 for x in range(classes_count)] for y in range(m)]
    for (_, pred_proba) in predictions:
        for i in range(m):
            for j in range(classes_count):
                results[i][j] += pred_proba[i][j]
    for i in range(m):
        for j in range(classes_count):
            results[i][j] /= n_classifiers            
    return results

def majority_pred(predictions):
    m = len(predictions[0][0])
    results = [0] * m
    for i in range(m):
        all_results = [0 for x in range(classes_count)] 
        for (pred, _) in predictions:
            all_results[pred[i]] += 1
        results[i] = all_results.index(max(all_results))        
    return results

def majority_pred_proba(predictions, n_classifiers):
    m = len(predictions[0][0])
    results = [[0 for x in range(classes_count)] for y in range(m)]
    classifiers_votes_count = [0] * m
    majority_results = majority_pred(predictions)
    for classifier in range(0, n_classifiers):
        for i in range(m):
            voted_class = majority_results[i] 
            (pred, pred_proba) = predictions[classifier]
            if(pred[i] == voted_class):
                classifiers_votes_count[i] += 1
                for j in range(classes_count):
                    results[i][j] += pred_proba[i][j]
    for i in range(m):
        for j in range(classes_count):
            results[i][j] /= classifiers_votes_count[i]
    return results      
    

def borda_pred(predictions):
    m = len(predictions[0][0])
    all_results = [[0 for x in range(classes_count)] for y in range(m)] 
    results = [0] * m
    
    def get_final_borda_points(predictions):
        return np.argsort(np.argsort(predictions)).tolist()

    for (_, pred_proba) in predictions:
        for i in range(m):
            pred_proba[i] = get_final_borda_points(pred_proba[i])
    for (_, pred_proba) in predictions:
        for i in range(m):
            for j in range(classes_count):
                all_results[i][j] += pred_proba[i][j]
    for i in range(m):
        results[i] = all_results[i].index(max(all_results[i]))
    return results

def borda_pred_proba(predictions, n_classifiers):
    m = len(predictions[0][0])
    results = [[0 for x in range(classes_count)] for y in range(m)] 
    
    def get_final_borda_points(predictions):
        return np.argsort(np.argsort(predictions)).tolist()

    def get_points():
        sum = 0
        for i in range(classes_count):
            sum += i
        return sum * n_classifiers
    
    for (_, pred_proba) in predictions:
        for i in range(m):
            pred_proba[i] = get_final_borda_points(pred_proba[i])
    for (_, pred_proba) in predictions:
        for i in range(m):
            for j in range(classes_count):
                results[i][j] += pred_proba[i][j]
    for i in range(m):
        for j in range(classes_count):
            results[i][j] /= get_points()
        
    return results

In [16]:
res = majority_pred(clas)

In [17]:
accuracy = accuracy_score(y_test, res)

In [18]:
accuracy

0.6741901221455124

In [19]:
def make_noise_label(labels, percent):
    labels_with_noise = deepcopy(labels)
    arr_size = labels.shape[0]
    indexes = [n for n in range(arr_size)]
    indexes_to_change = sample(indexes, int(arr_size * percent))
    
    for i in indexes_to_change:
        old_val = labels[i]
        new_val = randint(0, classes_count)
        while old_val == new_val:
            new_val = randint(0, classes_count)
        labels_with_noise[i] = new_val
        
    return labels_with_noise

In [20]:
def make_noise_data(data, percent):
    m = data.shape[0]
    n = data.shape[1]
    new_data = deepcopy(data)
    for i in range(m):
        for j in range(n):
            rand = randrange(-1, 2, 2) #random integer from {-1, 1}
            new_data[i][j] = (1 + rand * percent) * data[i][j]
    return new_data

In [114]:
clf_100iter = SVC(probability=True, max_iter=50) #221 dla 100, 24 dla 10, 46 dla 20, 112 dla 50

with elapsed_timer() as t:
    clf_100iter.fit(svd_x_train, y_train)

print("duration: " + str(t.duration))

duration: 111.54180649999944


C:\Users\majer\Anaconda3\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=50).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Wyniki jakości klasyfikatorów:

In [193]:
clf = MinorClassifiers(n_samples, n_features)

In [194]:
clf.fit(svd_x_train, y_train)

In [195]:
pred = clf.predict(svd_x_test)

In [191]:
clf.predictions[2]

(array([ 1, 12,  0, ...,  9,  3, 15]),
 array([[9.14988551e-03, 2.15277112e-01, 1.73763115e-02, ...,
         6.70366836e-03, 1.56338312e-02, 3.31547907e-02],
        [3.56125316e-03, 1.96303773e-01, 5.98553948e-02, ...,
         6.58740666e-04, 1.61431898e-03, 7.39091693e-03],
        [9.05976285e-01, 5.28840577e-03, 7.41057913e-03, ...,
         7.06449449e-03, 1.90014454e-03, 3.77637276e-02],
        ...,
        [2.35469869e-02, 1.83868394e-02, 1.75002204e-02, ...,
         6.27684865e-03, 2.37346573e-02, 9.10012140e-02],
        [4.52282917e-04, 4.66485262e-02, 3.33162473e-02, ...,
         4.09182284e-04, 7.68833038e-04, 2.72883193e-03],
        [5.15397803e-02, 9.25186881e-04, 7.47807866e-04, ...,
         1.92568251e-03, 3.02629863e-03, 8.29185369e-02]]))

In [175]:
pred[82]

12

In [196]:
pred_proba = clf.predict_proba(svd_x_test)

In [13]:
scores = cross_validate(clf, svd_x_train, y_train, cv=5, scoring=('accuracy', 'neg_log_loss'))

In [14]:
print(scores['test_accuracy'])
print(scores['test_neg_log_loss'])

[0.75828546 0.75828546 0.76535572 0.76844896 0.74093722]
[-1.00897419 -1.04675676 -1.00527875 -1.05143864 -1.09940959]


In [197]:
#UWAGA NIE PUSZCZAĆ BEZ POWODU - linijka niezakomentowana żeby uniknąć restartowania kernela
#cross-validation -> wybór metody fuzji decyzji?
scores = cross_validate(clf, svd_x_train, y_train, cv=2, scoring=('accuracy', 
                                                                  'neg_log_loss', 
                                                                  'precision_weighted',
                                                                  'recall_weighted',
                                                                  'f1_weighted', 
                                                                  'roc_auc_ovr')) #zrobić klasę Model, robiącą model z 10 klasyfikatorów
print(scores['test_accuracy'])
print(scores['test_neg_log_loss'])
print(scores['test_precision_weighted'])
print(scores['test_recall_weighted'])
print(scores['test_f1_weighted'])
print(scores['test_roc_auc_ovr'])
#print(scores['test_average_precision'])

[0.69754287 0.65582464]
[-1.23190468 -1.36502393]
[0.75995394 0.74425157]
[0.69754287 0.65582464]
[0.71079543 0.66364172]
[0.96239758 0.95221274]


[0.99181438 0.99098517]


In [123]:
y_pred = clf.predict(svd_x_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy score: {0:0.2f}'.format(accuracy))

Accuracy score: 0.79


In [121]:
y_pred_proba = clf.predict_proba(svd_x_test)
loss = log_loss(y_test, y_pred_proba)
print('Loss score: {0:0.2f}'.format(loss))

Loss score: 0.72


In [129]:
average_precision = average_precision_score(y_test, y_pred, pos_label=1)
print('Precision-recall score: {0:0.2f}'.format(average_precision))

ValueError: multiclass format is not supported

In [15]:
x_train

<11314x130107 sparse matrix of type '<class 'numpy.float64'>'
	with 1787565 stored elements in Compressed Sparse Row format>

In [7]:
a = x_train.toarray()

In [6]:
b = np.array([[2.0,3.0], [2.0,4.], [4.,5.]])

In [17]:
c = make_noise_data(svd_x_train, 0.1)
print(c)

[[ 0.22475435 -0.06248808 -0.01441603 ... -0.0168442   0.02101432
  -0.0191214 ]
 [ 0.12599262 -0.08438557 -0.0357827  ... -0.01513339  0.00517915
   0.02194321]
 [ 0.3346583  -0.04556982 -0.07380376 ...  0.00602186  0.01392154
  -0.02985655]
 ...
 [ 0.1662913  -0.00550221 -0.0884303  ...  0.01776318 -0.00348805
   0.01692393]
 [ 0.20675388 -0.07267247  0.03741476 ...  0.03050179  0.02278233
  -0.02552414]
 [ 0.09054867 -0.09988643  0.00410151 ...  0.00048173 -0.02367268
   0.00355212]]
